In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB
%matplotlib inline
from sklearn.naive_bayes import BernoulliNB
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [76]:
df1 = pd.read_excel('titanic.xls')
df1.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [77]:
df1.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')

In [78]:
df=df1[['pclass','survived','sex','age','fare']]

In [79]:
df

,pclass,survived,sex,age,fare
0,1,1,female,29.0000,211.3375
1,1,1,male,0.9167,151.5500
2,1,0,female,2.0000,151.5500
3,1,0,male,30.0000,151.5500
4,1,0,female,25.0000,151.5500
...,...,...,...,...,...
1304,3,0,female,14.5000,14.4542
1305,3,0,female,NaN,14.4542
1306,3,0,male,26.5000,7.2250
1307,3,0,male,27.0000,7.2250


In [80]:
df["age"].fillna(df.groupby(['pclass','survived','sex'])["age"].transform('mean'), inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [81]:
len(df)

1309

In [82]:
df['sex'] = df['sex'].map({'female':0, 'male':1})

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [83]:
df=df.dropna(how='any')

In [84]:
df.isnull().sum()

pclass      0
survived    0
sex         0
age         0
fare        0
dtype: int64

In [85]:
len(df)

1308

In [86]:
df.head()

,pclass,survived,sex,age,fare
0,1,1,0,29.0000,211.3375
1,1,1,1,0.9167,151.5500
2,1,0,0,2.0000,151.5500
3,1,0,1,30.0000,151.5500
4,1,0,0,25.0000,151.5500


In [87]:
target=df.survived
inputs=df.drop('survived',axis='columns')

In [88]:
inputs.columns[inputs.isna().any()]

Index([], dtype='object')

In [89]:
inputs.age[:10]

0    29.0000
1     0.9167
2     2.0000
3    30.0000
4    25.0000
5    48.0000
6    63.0000
7    39.0000
8    53.0000
9    71.0000
Name: age, dtype: float64

In [90]:
inputs.age=inputs.age.fillna(inputs.age.mean())

In [91]:
inputs.head()

,pclass,sex,age,fare
0,1,0,29.0000,211.3375
1,1,1,0.9167,151.5500
2,1,0,2.0000,151.5500
3,1,1,30.0000,151.5500
4,1,0,25.0000,151.5500


In [92]:
X_train, X_test, y_train, y_test = train_test_split(inputs,target, test_size=0.2)

In [93]:
len(X_train)

1046

In [94]:
len(X_test)

262

In [95]:
len(inputs)

1308

In [96]:
X_train

,pclass,sex,age,fare
1133,3,1,17.000000,8.6625
870,3,0,27.000000,7.9250
277,1,0,37.109375,146.5208
827,3,1,11.000000,46.9000
643,3,0,5.000000,31.3875
...,...,...,...,...
1240,3,1,0.416700,8.5167
879,3,1,26.679598,7.8958
319,1,0,31.000000,134.5000
1222,3,1,33.000000,8.6625


 Gaussian

In [97]:
gnb = GaussianNB()

In [98]:
gnb.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [100]:
gnb.score(X_train, y_train)

0.7552581261950286

In [101]:
y_pred = gnb.predict(X_test)


In [102]:
conf= pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Failed', 'Predicted Passed'],
    index=['True did not survived', 'True survived']
)

conf

,Predicted Failed,Predicted Passed
True did not survived,134,31
True survived,20,77


In [103]:
y_test.value_counts()

0    165
1     97
Name: survived, dtype: int64

In [104]:
gnb.score(X_test, y_test)

0.8053435114503816

In [105]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.81      0.84       165
           1       0.71      0.79      0.75        97

    accuracy                           0.81       262
   macro avg       0.79      0.80      0.80       262
weighted avg       0.81      0.81      0.81       262



 Bernoulli Naive Bayes

In [108]:
bnb = BernoulliNB()

In [109]:
bnb.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [110]:
bnb.score(X_train, y_train)

0.7705544933078394

In [111]:
y_pred = gnb.predict(X_test)

In [113]:
cnfbnb = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Failed', 'Predicted Passed'],
    index=['True did not survived', 'True survived']
)

cnfbnb

,Predicted Failed,Predicted Passed
True did not survived,134,31
True survived,20,77


In [114]:
gnb.score(X_test, y_test)

0.8053435114503816

In [118]:
y_predict = bnb.predict(X_test)

In [119]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.85      0.87      0.86       165
           1       0.76      0.73      0.75        97

    accuracy                           0.82       262
   macro avg       0.80      0.80      0.80       262
weighted avg       0.82      0.82      0.82       262

